# Second Order: Cantilever Colum
In this example, we demonstrate the difference between a First-Order and a Second-Order structural analysis of a cantilever column.
The example shows how the system is defined, how the solver works internally, and how geometric nonlinear effects are considered.

**System description**

We consider a 4 m high cantilever column with a HEA 240 profile.
The column is subjected to a concentrated load at the free end and a trapezoidal distributed load along its length.

- Length: 4 m
- Cross-section: HEA 240
    - $I$ = 2769 cm$^4$
    - $A$ = 76.84 cm$^2$
    - $\kappa$ = 0.6275377 
- Material: S 235 (Steel)
    - $E$ = 21 000 $\frac{\text{kN}}{\text{cm}^2}$
    - $G$ = 8 100 $\frac{\text{kN}}{\text{cm}^2}$
- Loads:
    - Point load at the top: $P$ = 182 kN
    -  Trapezoidal distributes load over the entire length

<p align="center">
  <img src="second_order_system.png" alt="Plot" width=35%>
</p>

## Import Modules 

In [1]:
from sstatics.core.preprocessing import Bar, BarLineLoad, CrossSection, Material, Node, NodePointLoad, System
from sstatics.core.calc_methods import FirstOrder, SecondOrder

## Define Cross-Section and Material
Instead of creating a geometry based on a polygonal shape, we can directly specify the **stiffness values** for the cross-section. 

In [2]:
c_1 = CrossSection(0.00002769, 0.007684, 0.2, 0.2, 0.6275377)

Next we define the **material properties**:

In [3]:
m_1 = Material(210000000, 0.1, 81000000, 0.1)

## Define Nodes
We create the system’s **nodes**, which represent the joints of the structure in the global coordinate system. Supports or boundary conditions also be applied here.

In [4]:
node_1 = Node(x=0, z=0, u='fixed', w='fixed', phi='fixed')
node_2 = Node(x=0, z=-4, loads=NodePointLoad(x=0, z=182, phi=0, rotation=0))

## Define Bar
The nodes are then connected by **bar elements**. Each bar is assigned the previously defined cross-section and material properties.

In [5]:
bar_1 = Bar(node_1, node_2, c_1, m_1, line_loads=BarLineLoad(pi=1, pj=1.5, direction='z', coord='bar', length='exact'))

## Define System
The bars are assembled into a **structural system object**.

In [6]:
system = System([bar_1])

## First-Order Analysis
In the first step, the system is solved according to First-Order Theory (linear analysis). This step provides for example displacements and internal forces without considering geometric nonlinearities. To see a detailled list of all the possible results of the First-Order Analysis go to API References of `FirstOrder`. The displacements and internal forces are obtained for small deformations. These serve as the basis for the Second-Order analysis.

In [7]:
# Get Solution Instance
solution_first_order = FirstOrder(system)

bar_deform_first = solution_first_order.bar_deform_list
forces_first = solution_first_order.internal_forces

## Transition to Second-Order Theory
The Second-Order Theory accounts for geometric nonlinear effects, such as P–Δ effects.
The computational procedure is as follows:

1. Initial Solution:
   - The system is first solved using the standard `Solver` to obtain the internal forces and deformations of the bar, which are important to calculate the average axial force.

2. Average Axial Force:
    - The average longitudinal (axial) force in each bar is computed by the use of the method `averaged_longitudinal_force`.
    - This force is used to modify the stiffness matrices and load vectors.

3. Bar Conversion:
    - Using the method `_convert_bars`, each bar of class `Bar` is converted into a `BarSecondOrder`.
    - These new elements contain the nonlinear geometric terms.

4. System Assembly
    - The converted `BarSecondOrder`elements are assembled into a new system.
    - The topology remains the same, but now the system includes geometric nonlinear behavior.
      
5. System Solution
    - The new system can be solved again using the `Solver`Class
  
To simplify the workflow, dedicated methods are provided to create `Solver` objects according to the selected approach.
To perform the calculation using the matrix-based approach, call `matrix_approach` and select a calculation variant. The available variants are:
- `analytic`: Analytical matrix formulation
- `taylor`: Taylor series approximation
- `p_delta`: Simplified P–Δ approach

In [8]:
# Create a SecondOrder Object
sec_order = SecondOrder(system)

# Select and Run Second-Order for 'analytic' variant
sec_order.matrix_approach('analytic')

After calling `matrix_approach`, you can access the `Solver` via `solver_matrix_approach` or the modified system via `system_matrix_approach`.
These methods are useful for inspecting local stiffness matrices and to get internal forces and deformations according to Second-Order Theory.
The system or solver instance cannot be accessed before calling `matrix_approach`. If a different calculation variant is selected, the results from the previous variant are discarded, and the new approach is applied for both `system_matrix_approach` and `solver_matrix_approach`.

## Transformation of Internal Forces
The internal forces returned from the solver (`internal_forces`) are expressed in equivalent longitudinal and transverse components.
To obtain the true normal and shear forces, they must be transformed back using the method `_transform_internal_forces()`.
If you are already using the `solver_matrix_approach` function, the transformation of the forces has already accored.

## Comparison of First and Second-Order Results
Finally, the difference between First-Order and Second-Order results is analyzed:
- Differences in displacements
- Differences in bending moments and axial forces
- Visualization of geometric nonlinear effects

### Comparing the bending moment at the beginning of the bar

In [9]:
# Solution analytic
solution_analytic = sec_order.solver_matrix_approach

# Solution taylor
sec_order.matrix_approach('taylor')
solution_taylor = sec_order.solver_matrix_approach

# Solution p-delta
sec_order.matrix_approach('p_delta')
solution_p_delta = sec_order.solver_matrix_approach

# Comparing the internal forces of the bending moment at the beginning of the bar
print('Mi - first order: [kNm]', forces_first[0][2][0])
print('Mi - second order analytic: [kNm]', solution_analytic.internal_forces[0][2][0])
print('Mi - second order taylor: [kNm]', solution_taylor.internal_forces[0][2][0])
print('Mi - second order p delta: [kNm]', solution_p_delta.internal_forces[0][2][0])

Mi - first order: [kNm] 10.666666666666709
Mi - second order analytic: [kNm] 12.369294641439046
Mi - second order taylor: [kNm] 12.373174522851505
Mi - second order p delta: [kNm] 12.30973930064128


### Comparing the bar deformation at the end of the cantilever in w - direction

In [10]:
# Comparing the bar deformation at the end of the cantilever
print('w - first order: [m]', bar_deform_first[0][4][0])
print('w - second order analytic: [m]', solution_analytic.bar_deform_list[0][4][0])
print('w - second order taylor: [m]', solution_taylor.bar_deform_list[0][4][0])
print('w - second order p delta: [m]', solution_p_delta.bar_deform_list[0][4][0])

w - first order: [m] 0.00752088601694359
w - second order analytic: [m] 0.009362070985367796
w - second order taylor: [m] 0.009383463808579719
w - second order p delta: [m] 0.009027885973583321


> ⚠️ **Note:**  
> It should be noted that the sign convention is determined according to the deformation method.

## Conclusion
The **Second-Order Theory** enables the consideration of geometric nonlinearities and provides more realistic results, especially for slender structural members.
The use of `BarSecondOrder` and the corresponding solvers allows flexible and accurate structural analysis.